In [ ]:
%matplotlib notebook
import numpy as np
from scipy.optimize import minimize
import random as rd
import matplotlib.pyplot as plt
from brainsignals.plotting_convention import mark_subplots
np.random.seed(1234)

In [ ]:
# generate disk electrode with orientation
# =======================================================================
# Input parameters:
# =======================================================================
# N            : 3D normal vector to define orientation of disk electrode
# c0           : center of disk
# n            : number of points to constitute the disk
# disk_radius  : radius of disk
# =======================================================================
# Outputs     : C, a numpy array of point coordinates with shape (n,3) 

def generate_disk_points(N, c0, n, disk_radius):
    
    x = rd.uniform(0,1)
    y = rd.uniform(0,1)
    z = rd.uniform(0,1)
    a = np.array([x,y,z])
    alfa = np.cross(N,(N+a))/np.linalg.norm(N+a)
    beta = np.cross(N,alfa)
    alfa_n = alfa/np.linalg.norm(alfa)
    beta_n = beta/np.linalg.norm(beta)
    
    i = 0
    C = np.zeros(shape=(n, 3))
    r_max = 0
    
    while i<n:
        u = rd.uniform(-disk_radius,disk_radius)
        v = rd.uniform(-disk_radius,disk_radius)
        if (u**2+v**2) < disk_radius**2:
            C[i] = u*alfa_n + v*beta_n + c0
            i += 1
            if (u**2+v**2) > r_max**2:
                r_max = np.sqrt(u**2+v**2)
    return C

In [ ]:
def gen_random_disk_set(n_elecs, disk_radius, n, x0, x1):
    N_vecs = np.array([(1,0,0), (0,1,0), (0,0,1), (-1,0,0), (0,-1,0), (0,0,-1)])

    k= 0
    first = True
    for i in range(n_elecs):
        k = np.random.randint(6)
        N = N_vecs[k]
        c0 = np.random.uniform(x0, x1, size=3)
        el_coords1 = generate_disk_points(N, c0, n, disk_radius)
        if first:
            elec_coords=el_coords1
            first = False
        else:
            newrow = el_coords1
            elec_coords = np.vstack([elec_coords, newrow])
        k+=1
    return elec_coords

In [ ]:
# generate set of disk electrodes

def gen_disk_set(n_elecs_set, disk_radius, set_width, n, x0, x1):
    
    normal_vec_set = np.array([(1,0,0),(-1,0,0), (0,1,0), (0,-1,0)])
    set_radius = set_width/2
    k = 0
    # calc coordinates for disk electrode sets
    first = True
    for i in range(n_elecs_set):
        pos_elec_set = np.random.uniform(x0, x1, size=3)
        for j in range(4):
            center = pos_elec_set + set_radius*normal_vec_set[j]
            N = normal_vec_set[j]
            el_coords1 = generate_disk_points(N, center, n, disk_radius)
            if first:
                elec_coords=el_coords1
                first = False
            else:
                
                newrow = el_coords1
                elec_coords = np.vstack([elec_coords, newrow])
            k+=1
            
    return elec_coords

In [ ]:
# generate cylinder electrode 
# =======================================================================
# Input parameters:
# =======================================================================
# start_z       : starting point of cylinder height   
# end_z         : ending point of cylinder height 
# mid_x         : midpoint x-axis
# mid_y         : midpoint y-axis
# radius        : cylinder radius
# num_points    : number of points to constitute the cylinder
# test_scale    : parameter for testing, default as 1
# =======================================================================
# Outputs       : C, a numpy array of coordinates with shape (num_points,3) 


def gen_sylinder_points(start_z, end_z, mid_x, mid_y, radius, num_points, test_scale):
    
    z_ = np.random.uniform(test_scale*start_z, test_scale*end_z, num_points)
    theta = np.random.uniform(0, 2*np.pi, num_points)
    x_ = mid_x + radius*test_scale*np.cos(theta)
    y_ = mid_y + radius*test_scale*np.sin(theta)
    C = np.array([x_, y_, z_])
    
    return C.T

In [ ]:
# generate cylinder elec set w/ rand pos inside box
def gen_sylinder_set(n_elecs, num_points, syl_height, syl_width, x0, x1, test_scale):
    
    k = 0
    first = True
    for i in range(n_elecs):
        start_z = np.random.uniform(x0, x1)
        end_z = start_z + syl_height
        mid_x = np.random.uniform(x0, x1)
        mid_y = np.random.uniform(x0, x1)
        el_coords1 = gen_sylinder_points(start_z, end_z, mid_x, mid_y, (syl_width/2), num_points, test_scale)
        
        if first:
            elec_coords=el_coords1
            first = False
        else:
            newrow = el_coords1
            elec_coords = np.vstack([elec_coords, newrow])
        k+=1
    return elec_coords

In [ ]:
# gen set of disk set elecs with set posititions, also return center locs of disks
def g_d_s(disk_radius, set_radius, n_points, n_elecs_set, center_pos):
    normal_vec_set = np.array([(1,0,0),(-1,0,0), (0,1,0), (0,-1,0)])
    centers = []
    k = 0
    first = True
    for i in range(n_elecs_set):
        pos_elec_set = center_pos[i]
        for j in range(4):
            center = pos_elec_set + set_radius*normal_vec_set[j]
            N = normal_vec_set[j]
            el_coords1 = generate_disk_points(N, center, n_points, disk_radius)
            centers.append(center)
            if first:
                elec_coords=el_coords1
                first = False
            else:
                newrow = el_coords1
                elec_coords = np.vstack([elec_coords, newrow])
                
            k+=1
            
    return elec_coords, centers

In [ ]:
# gen set of sylinder elecs with set posititions

def g_s_s(n_elecs, n_points, syl_height, syl_width, center_locs, test_scale):
    
    k = 0
    first = True
    for i in range(n_elecs):
        mid_z = center_locs[i][2]
        start_z = mid_z - (syl_height/2)
        end_z = start_z + syl_height
        mid_x = center_locs[i][0]
        mid_y = center_locs[i][1]
        el_coords1 = gen_sylinder_points(start_z, end_z, mid_x, mid_y, (syl_width/2), n_points, test_scale)
        if first:
            elec_coords=el_coords1
            first = False
        else:
            newrow = el_coords1
            elec_coords = np.vstack([elec_coords, newrow])
        k+=1
        
    return elec_coords

In [ ]:
# plot set of spatially extended electrodes 
def plot_elecs(elec_coords, dipole_pos):
    X = []
    Y = []
    Z = []
    for i in range(len(elec_coords)):
        X.append(elec_coords[i][0])
        Y.append(elec_coords[i][1])
        Z.append(elec_coords[i][2])

    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_aspect(aspect='auto')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.scatter(X,Y,Z)
    ax.scatter(dipole_pos[0], dipole_pos[1], dipole_pos[2])
    
    return ax

In [ ]:
def plot_res(elec_coords, dipole_pos, opt, filename, title):
    X = []
    Y = []
    Z = []
    for i in range(len(elec_coords)):
        X.append(elec_coords[i][0])
        Y.append(elec_coords[i][1])
        Z.append(elec_coords[i][2])

    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_aspect(aspect='auto')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.scatter(X,Y,Z)
    ax.scatter(dipole_pos[0], dipole_pos[1], dipole_pos[2])
    ax.scatter(opt[0], opt[1], opt[2], color = 'red')
    fig.suptitle(title, fontsize=10)
    fig.savefig(filename, bbox_inches='tight')
    return ax

In [ ]:
def V_e_avg(V_e, n_points, n_elecs):
    # averages over electrode surface
    
    V_e_avg = np.zeros(shape=(n_elecs, ))
    
    for i in range(n_elecs):
        V_e_avg[i] = np.sum(V_e[n_points*(i):n_points*(i+1)])/n_points
    
    return V_e_avg

In [ ]:
# method for calculating extracellular potential through dipole approximation
# ===========================================================================
# Input parameters:
# ===========================================================================
# x           :  3D coordinates of dipole
# elec_locs   :  coordinates for points constituting the electrode surfaces
# p           :  dipole moment
# n_points    :  number of points on each electrode surface
# n_elecs     :  number of electrode surfaces
# ===========================================================================
# Outputs     : average extracellular potential for each electrode surface
# ===========================================================================

sigma = 0.3  # S/m
def dipole_potential(x, elec_locs, p, n_points, n_elecs):
    
    r_ = elec_locs - x
    V_e = r_ @ p.T / (4 * np.pi * sigma * np.linalg.norm(r_, axis=1) ** 3)
    
    # averaging to account for spatial extention of electrodes
    avg = V_e_avg(V_e, n_points, n_elecs)
    
    return avg

In [ ]:

def dipole_potential_min(test_pos, elec_locs, V_e_measured, p, n_points, n_elecs):
    
    V_e = dipole_potential(test_pos, elec_locs, p, n_points, n_elecs)
    res = (V_e-V_e_measured) /((np.sqrt(np.abs(V_e_measured*V_e))))    # Normalizing seemed to help
    
    return np.sum(res**2)

In [ ]:
# sim 1
# comparing pointelecs, sylinders and disk sets with random electrode positions
orig_p = np.array([1., 1., 1.]) * 1e8
num_trials = 1000
# no of electrodes generated
n_elecs_syl = 5
# no of disk sets
n_elecs_set = 5 
# total no of electrodes generated in disk sets
n_elecs_disk = n_elecs_set*4
n_elecs_point = n_elecs_set*4
# no of points to average over on each electrode surface
n_points_point = 1
n_points_syl = 1000
n_points_disk = 1000
# electrode sizes
syl_height = 2000
syl_width = 2000
test_scale = 1
disk_radius = 500
set_width = 2000

point_errors1 = np.zeros(num_trials)
point_fails1 = []
syl_errors1 = np.zeros(num_trials)
syl_fails1 = []
disk_errors1 = np.zeros(num_trials)
disk_fails1 = []

box_width = 10000
x0 = -box_width / 2
x1 = box_width / 2


for trial_idx in range(num_trials):
    orig_loc = np.random.uniform(x0, x1, size=(3))
    point_elec_locs  = np.random.uniform(x0, x1, size=(n_elecs_point, 3))
    V_e_measured_point = dipole_potential(orig_loc, point_elec_locs, orig_p, n_points_point, n_elecs_point)
    
    retry = True
    counter = 0
    while retry:
        
        dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)
        point_opt = minimize(dipole_potential_min, dipole_pos0,
                        options = {'maxiter': 50000., 'maxls': 100, 'gtol': 1e-12},
                        method='L-BFGS-B',
                        args =(point_elec_locs, V_e_measured_point, orig_p, n_points_point, n_elecs_point), 
                        bounds=((x0*1.2, x1*1.2), (x0*1.2, x1*1.2), (x0*1.2, x1*1.2)), tol=1e-12
                        )
        V_e_best_guess_point = dipole_potential(point_opt.x, point_elec_locs, orig_p, n_points_point, n_elecs_point)
        rel_error_point = np.mean(np.abs((V_e_measured_point - V_e_best_guess_point) / V_e_measured_point))

        if (rel_error_point < 0.05) or (counter > 100):
            # If it doesn't work well, we just retry with a different initial guess
            retry = False

        counter += 1
    point_errors1[trial_idx] = np.sqrt(np.sum((orig_loc - point_opt.x)**2))
    if point_errors1[trial_idx] > 1000:
        point_fails1.append([point_elec_locs, orig_loc, point_errors1[trial_idx], point_opt.x, V_e_best_guess_point, 
                            V_e_measured_point, dipole_pos0, point_opt])
    
for trial_idx in range(num_trials):
    orig_loc = np.random.uniform(x0, x1, size=(3))
    syl_elec_locs = gen_sylinder_set(n_elecs_syl, n_points_syl, syl_height, syl_width, x0, x1, test_scale)
    V_e_measured_syl = dipole_potential(orig_loc, syl_elec_locs, orig_p, n_points_syl, n_elecs_syl)
    
    retry = True
    syl_counter = 0
    
    while retry:
        dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)
        syl_opt = minimize(dipole_potential_min, dipole_pos0,
                         options = {'maxiter': 50000., 'maxls': 100, 'gtol': 1e-12},
                         method='L-BFGS-B',
                         args =(syl_elec_locs, V_e_measured_syl, orig_p, n_points_syl, n_elecs_syl), 
                         bounds=((x0*1.2, x1*1.2), (x0*1.2, x1*1.2), (x0*1.2, x1*1.2)), tol=1e-12
                         )
        
        V_e_best_guess_syl = dipole_potential(syl_opt.x, syl_elec_locs, orig_p, n_points_syl, n_elecs_syl)
        rel_error_syl = np.mean(np.abs((V_e_measured_syl - V_e_best_guess_syl) / V_e_measured_syl))
        
        if (rel_error_syl < 0.05) or (syl_counter > 100):
            # If it doesn't work well, we just retry with a different initial guess
            retry = False

        syl_counter += 1
    syl_errors1[trial_idx] = np.sqrt(np.sum((orig_loc - syl_opt.x)**2))
    if syl_errors1[trial_idx] > 1000:
        syl_fails1.append([syl_elec_locs, orig_loc, syl_errors1[trial_idx], syl_opt.x, V_e_best_guess_syl, 
                            V_e_measured_syl, dipole_pos0, syl_opt])

for trial_idx in range(num_trials):
    orig_loc = np.random.uniform(x0, x1, size=(3))
    disk_elec_locs  = gen_disk_set(n_elecs_set, disk_radius, set_width, n_points_disk, x0, x1)
    V_e_measured_disk = dipole_potential(orig_loc, disk_elec_locs, orig_p, n_points_disk, n_elecs_disk)
    
    retry = True
    disk_counter = 0
    while retry:
        dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)
        disk_opt = minimize(dipole_potential_min, dipole_pos0,
                        options = {'maxiter': 50000., 'maxls': 100, 'gtol': 1e-12},
                        method='L-BFGS-B',
                        args =(disk_elec_locs, V_e_measured_disk, orig_p, n_points_disk, n_elecs_disk), 
                        bounds=((x0*1.2, x1*1.2), (x0*1.2, x1*1.2), (x0*1.2, x1*1.2)), tol=1e-12
                        )
        V_e_best_guess_disk = dipole_potential(disk_opt.x, disk_elec_locs, orig_p, n_points_disk, n_elecs_disk)
        rel_error_disk = np.mean(np.abs((V_e_measured_disk - V_e_best_guess_disk) / V_e_measured_disk))

        if (rel_error_disk < 0.05) or (disk_counter > 100):
                # If it doesn't work well, we just retry with a different initial guess
            retry = False

        disk_counter += 1
    disk_errors1[trial_idx] = np.sqrt(np.sum((orig_loc - disk_opt.x)**2))
    if disk_errors1[trial_idx] > 1000:
        disk_fails1.append([disk_elec_locs, orig_loc, disk_errors1[trial_idx], disk_opt.x, V_e_best_guess_disk, 
                            V_e_measured_disk, dipole_pos0, disk_opt])
            
print(f"Mean error(point): {np.mean(point_errors1)}, Median error(point): {np.median(point_errors1)}")
fig = plt.figure(figsize=[4, 4], dpi=100)
plt.ylabel("count #")
plt.xlabel("point error (µm)")
h = plt.hist(point_errors1, bins=20)
fig.suptitle('Point errors (µm)', fontsize=10)
fig_name = f'point error histogram'
fig_folder = 'sim_1_rerun_tot'
fig.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')


print(f"Mean error(syl): {np.mean(syl_errors1)}, Median error(syl): {np.median(syl_errors1)}")
fig = plt.figure(figsize=[4, 4], dpi=100)
plt.ylabel("count #")
plt.xlabel("sylinder error (µm)")
h = plt.hist(syl_errors1, bins=20)
fig.suptitle('Sylinder errors (µm)', fontsize=10)
fig_name = f'syl error histogram'
fig_folder = 'sim_1_rerun_tot'
fig.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')


print(f"Mean error(disk): {np.mean(disk_errors1)}, Median error(disk): {np.median(disk_errors1)}")
fig = plt.figure(figsize=[4, 4], dpi=100)
plt.ylabel("count #")
plt.xlabel("disk error (µm)")
h = plt.hist(disk_errors1, bins=20)
fig.suptitle('Disk errors (µm)', fontsize=10)
fig_name = f'disk error histogram'
fig_folder = 'sim_1_rerun_tot'
fig.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')


print('number of point fails > 1mm:', len(point_fails1))
for i in range(len(point_fails1)):
    print('error:', point_fails1[i][2])
    print(point_fails1[i][2], 'orig_loc:', point_fails1[i][1], 'opt_x:',point_fails1[i][3], 'init guess:',point_fails1[i][6],
          'V_e best guess:',point_fails1[i][4], 'V_e measured:',point_fails1[i][5], 'elec positions:', point_fails1[i][0])
    fig_name = f'point_fails1_{i+1}'
    fig_folder = 'sim_1_rerun_tot'
    plot_res(point_fails1[i][0], point_fails1[i][1], point_fails1[i][3],
            f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}',
            f'Point fail number {i+1}')

print('number of syl fails > 1mm:', len(syl_fails1))
for i in range(len(syl_fails1)):
    print('error:',syl_fails1[i][2])
    print(syl_fails1[i][2], 'orig_loc:', syl_fails1[i][1], 'opt_x:',syl_fails1[i][3], 'init guess:',syl_fails1[i][6],
          'V_e best guess:',syl_fails1[i][4], 'V_e measured:',syl_fails1[i][5], 'elec positions:', syl_fails1[i][0])
    fig_name = f'syl_fails1_{i+1}'
    fig_folder = 'sim_1_rerun_tot'
    plot_res(syl_fails1[i][0], syl_fails1[i][1], syl_fails1[i][3],
            f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}',
            f'Sylinder fail number {i+1}')

print('number of disk fails > 1mm:', len(disk_fails1))
for i in range(len(disk_fails1)):
    print('error:',disk_fails1[i][2])
    print(disk_fails1[i][2], 'orig_loc:', disk_fails1[i][1], 'opt_x:',disk_fails1[i][3], 'init guess:',disk_fails1[i][6],
          'V_e best guess:',disk_fails1[i][4], 'V_e measured:',disk_fails1[i][5], 'elec positions:', disk_fails1[i][0])
    fig_name = f'disk_fails1_{i+1}'
    fig_folder = 'sim_1_rerun_tot'
    plot_res(disk_fails1[i][0], disk_fails1[i][1], disk_fails1[i][3],
            f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}',
            f'Disk fail number {i+1}')

In [ ]:
# histograms
fig1, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[9,3], dpi=100)
ax1.hist(point_errors1, bins=20)
ax1.set_title('Point electrode errors')
ax1.tick_params('x', labelrotation=30)
ax1.set_ylabel('# count')
mark_subplots(ax1, 'A')
ax2.hist(syl_errors1, bins=20)
ax2.tick_params('x', labelrotation=30)
ax2.set_xlabel('error (µm)')
ax2.set_title('Cylinder electrode errors')
mark_subplots(ax2, 'B')
ax3.hist(disk_errors1, bins=20)
ax3.tick_params('x', labelrotation=30)
ax3.set_title('Disk electrode errors')
mark_subplots(ax3, 'C')

plt.tight_layout()

fig_folder = 'sim_1_rerun_tot'
fig_name = 'histograms_sim_1'
fig1.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')
print(f"Mean error(point): {np.mean(point_errors1)}, Median error(point): {np.median(point_errors1)}")
print(f"Mean error(syl): {np.mean(syl_errors1)}, Median error(syl): {np.median(syl_errors1)}")
print(f"Mean error(disk): {np.mean(disk_errors1)}, Median error(disk): {np.median(disk_errors1)}")
print('number of point fails > 1mm:', len(point_fails1))
print('number of cyl fails > 1mm:', len(syl_fails1))
print('number of disk fails > 1mm:', len(disk_fails1))

In [ ]:
# log histograms

fig1, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[9,3], dpi=100)
ax1.hist(point_errors1, bins=20)
ax1.set_yscale('log')
ax1.set_title('Point electrode errors')
ax1.tick_params('x', labelrotation=30)
ax1.set_ylabel('# count')
mark_subplots(ax1, 'A')
ax2.hist(syl_errors1, bins=20)
ax2.set_yscale('log')
ax2.tick_params('x', labelrotation=30)
ax2.set_xlabel('error (µm)')
ax2.set_title('Cylinder electrode errors')
mark_subplots(ax2, 'B')
ax3.hist(disk_errors1, bins=20)
ax3.set_yscale('log')
ax3.tick_params('x', labelrotation=30)
ax3.set_title('Disk electrode errors')
mark_subplots(ax3, 'C')

plt.tight_layout()

fig_folder = 'sim_1_rerun_tot'
fig_name = 'histograms_sim_1_log'
fig1.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')
print(f"Mean error(point): {np.mean(point_errors1)}, Median error(point): {np.median(point_errors1)}")
print(f"Mean error(syl): {np.mean(syl_errors1)}, Median error(syl): {np.median(syl_errors1)}")
print(f"Mean error(disk): {np.mean(disk_errors1)}, Median error(disk): {np.median(disk_errors1)}")
print('number of point fails > 1mm:', len(point_fails1))
print('number of cyl fails > 1mm:', len(syl_fails1))
print('number of disk fails > 1mm:', len(disk_fails1))

In [ ]:
np.savetxt('sim_1_point_errors1_rerun_tot.txt', point_errors1)
np.savetxt('sim_1_disk_errors1_rerun_tot.txt', disk_errors1)
np.savetxt('sim_1_syl_errors1_rerun_tot.txt', syl_errors1)

In [ ]:
#----------------------sim_2------------------------------
# comparing pointelecs, sylinders and disk sets with set electrode positions
orig_p = np.array([1., 1., 1.]) * 1e8
num_trials = 1000
# no of electrodes generated
n_elecs_syl = 5
# no of disk sets
n_elecs_set = 5 
# total no of disk- and pointelectrodes
n_elecs_disk = n_elecs_set*4
n_elecs_point = n_elecs_set*4
# no of points to average over each electrode surface
n_points_point = 1
n_points_syl = 1000
n_points_disk = 1000
# electrode sizes
syl_height = 2000
syl_width = 2000
test_scale = 1
disk_radius = 500
set_width = 2000 
set_radius = set_width/2
box_width = 10000
x0 = -box_width / 2
x1 = box_width / 2

elec_set_center_positions = np.array([(4000, -4000, -2000), (-4000, 4000, 1500), (4000, 4000, 4000),
                                      (-4000, -4000, 2000), (0, -4000, -4000)])
bounds = ((x0*1.2, x1*1.2), (x0*1.2, x1*1.2), (x0*1.2, x1*1.2))

# generate 5 sets of disk elecs and the centerpoints of each disk for point electrodes as referance
set_coords, cs = g_d_s(disk_radius, set_radius, n_points_disk, n_elecs_set, elec_set_center_positions)

n_elecs_syl = 5
syl_elec_locs = g_s_s(n_elecs_syl, n_points_syl, syl_height, syl_width, elec_set_center_positions, test_scale)

point_errors2 = np.zeros(num_trials)
syl_errors2 = np.zeros(num_trials)
disk_errors2 = np.zeros(num_trials)

point_fails2 = []
syl_fails2 = []
disk_fails2 = []


for trial_idx in range(num_trials):
    orig_loc = np.random.uniform(x0, x1, size=(3))
    point_elec_locs  = cs 
    V_e_measured_point = dipole_potential(orig_loc, point_elec_locs, orig_p, n_points_point, n_elecs_point)
    
    retry = True
    counter = 0
    while retry:
        
        dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)

        point_opt = minimize(dipole_potential_min, dipole_pos0,
                           options = {'maxiter': 50000., 'maxls':100, 'gtol': 1e-12},
                           method = 'L-BFGS-B',
                           args = (point_elec_locs, V_e_measured_point, orig_p, n_points_point, n_elecs_point),
                           bounds = bounds, tol = 1e-12)
        
        V_e_best_guess_point = dipole_potential(point_opt.x, point_elec_locs, orig_p, n_points_point, n_elecs_point)
        rel_error_point = np.mean(np.abs((V_e_measured_point - V_e_best_guess_point) / V_e_measured_point))

        if (rel_error_point < 0.05) or (counter > 100):
            # If it doesn't work well, we just retry with a different initial guess
            retry = False

        counter += 1

    point_errors2[trial_idx] = np.sqrt(np.sum((orig_loc - point_opt.x)**2))
    if point_errors2[trial_idx] > 1000:
        point_fails2.append([point_elec_locs, orig_loc, point_errors2[trial_idx], point_opt.x, V_e_best_guess_point, 
                            V_e_measured_point, dipole_pos0, point_opt])
    
for trial_idx in range(num_trials):
    orig_loc = np.random.uniform(x0, x1, size=(3))
    syl_elec_locs = syl_elec_locs#g_s_s(n_elecs_syl, n_points_syl, syl_height, syl_width, elec_set_center_positions, test_scale)
    V_e_measured_syl = dipole_potential(orig_loc, syl_elec_locs, orig_p, n_points_syl, n_elecs_syl)
    
    retry = True
    syl_counter = 0
    
    while retry:
        dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)

        syl_opt = minimize(dipole_potential_min, dipole_pos0,
                           options = {'maxiter': 50000., 'maxls':100, 'gtol': 1e-12},
                           method = 'L-BFGS-B',
                           args = (syl_elec_locs, V_e_measured_syl, orig_p, n_points_syl, n_elecs_syl),
                           bounds = bounds, tol = 1e-12)
        V_e_best_guess_syl = dipole_potential(syl_opt.x, syl_elec_locs, orig_p, n_points_syl, n_elecs_syl)
        rel_error_syl = np.mean(np.abs((V_e_measured_syl - V_e_best_guess_syl) / V_e_measured_syl))
        
        if (rel_error_syl < 0.05) or (syl_counter > 100):
            # If it doesn't work well, we just retry with a different initial guess
            retry = False

        syl_counter += 1
        
    syl_errors2[trial_idx] = np.sqrt(np.sum((orig_loc - syl_opt.x)**2))
    if syl_errors2[trial_idx] > 1000:
        syl_fails2.append([syl_elec_locs, orig_loc, syl_errors2[trial_idx], syl_opt.x, 
                          V_e_best_guess_syl, V_e_measured_syl, dipole_pos0, syl_opt])

for trial_idx in range(num_trials):
    orig_loc = np.random.uniform(x0, x1, size=(3))
    disk_elec_locs = set_coords
    V_e_measured_disk = dipole_potential(orig_loc, disk_elec_locs, orig_p, n_points_disk, n_elecs_disk)
    
    retry = True
    disk_counter = 0
    while retry:
        dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)

        disk_opt = minimize(dipole_potential_min, dipole_pos0,
                           options = {'maxiter': 50000., 'maxls':100, 'gtol': 1e-12},
                           method = 'L-BFGS-B',
                           args = (disk_elec_locs, V_e_measured_disk, orig_p, n_points_disk, n_elecs_disk),
                           bounds = bounds, tol = 1e-12)
        
        V_e_best_guess_disk = dipole_potential(disk_opt.x, disk_elec_locs, orig_p, n_points_disk, n_elecs_disk)
        rel_error_disk = np.mean(np.abs((V_e_measured_disk - V_e_best_guess_disk) / V_e_measured_disk))

        if (rel_error_disk < 0.05) or (disk_counter > 100):
                # If it doesn't work well, we just retry with a different initial guess
            retry = False

        disk_counter += 1

    disk_errors2[trial_idx] = np.sqrt(np.sum((orig_loc - disk_opt.x)**2))
    if disk_errors2[trial_idx] > 1000:
        disk_fails2.append([disk_elec_locs, orig_loc, disk_errors2[trial_idx], disk_opt.x, 
                           V_e_best_guess_disk, V_e_measured_disk, dipole_pos0, disk_opt])
            

print('number of point fails > 1 mm:', len(point_fails2))
for i in range(len(point_fails2)):
    print('error:', point_fails2[i][2])
    print(point_fails2[i][2], 'orig_loc:', point_fails2[i][1], 'opt_x:',point_fails2[i][3],'init guess:',point_fails2[i][6],
          'V_e best guess:',point_fails2[i][4], 'V_e measured:',point_fails2[i][5], 'elec positions:', point_fails2[i][0])
    fig_name = f'point_fails2_{i+1}'
    fig_folder = 'sim_2_rerun_tot'
    plot_res(point_fails2[i][0], point_fails2[i][1], point_fails2[i][3],
            f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}',
            f'Point fail number {i+1}')
    
print('number of syl fails > 1 mm:', len(syl_fails2))    
for i in range(len(syl_fails2)):
    print('error:',syl_fails2[i][2], 'orig_loc:',syl_fails2[i][1], 'opt_x:',syl_fails2[i][3] ,'init guess:',syl_fails2[i][6],
          'V_e best guess:',syl_fails2[i][4], 'V_e measured:',syl_fails2[i][5], 'elec positions:', syl_fails2[i][0])
    fig_name = f'syl_fails2_{i+1}'
    fig_folder = 'sim_2_rerun_tot'
    plot_res(syl_fails2[i][0], syl_fails2[i][1], syl_fails2[i][3],
            f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}',
            f'Sylinder fail number {i+1}')

print('number of disk fails > 1mm:', len(disk_fails2))
for i in range(len(disk_fails2)):
    print('error:',disk_fails2[i][2], 'orig_loc:', disk_fails2[i][1], 'opt_x:',disk_fails2[i][3], 'init guess:',disk_fails2[i][6],
          'V_e best guess:',disk_fails2[i][4], 'V_e measured:',disk_fails2[i][5], 'elec positions:', disk_fails2[i][0])
    fig_name = f'disk_fails2_{i+1}'
    fig_folder = 'sim_2_rerun_tot'
    plot_res(disk_fails2[i][0], disk_fails2[i][1], disk_fails2[i][3],
            f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}',
            f'Disk fail number {i+1}')

In [ ]:
# histograms sim_2
fig2, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[9,3], dpi=100)
ax1.hist(point_errors2, bins=20)
ax1.set_title('Point electrode errors')
ax1.tick_params('x', labelrotation=30)
ax1.set_ylabel('# count')
mark_subplots(ax1, 'A')
ax2.hist(syl_errors2, bins=20)
ax2.tick_params('x', labelrotation=30)
ax2.set_xlabel('error (µm)')
ax2.set_title('Cylinder electrode errors')
mark_subplots(ax2, 'B')
ax3.hist(disk_errors2, bins=20)
ax3.tick_params('x', labelrotation=30)
ax3.set_title('Disk electrode errors')
mark_subplots(ax3, 'C')

plt.tight_layout()

fig_folder = 'sim_2_rerun_tot'
fig_name = 'histograms_sim_2'
fig2.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')
print(f"Mean error(point): {np.mean(point_errors2)}, Median error(point): {np.median(point_errors2)}")
print(f"Mean error(syl): {np.mean(syl_errors2)}, Median error(syl): {np.median(syl_errors2)}")
print(f"Mean error(disk): {np.mean(disk_errors2)}, Median error(disk): {np.median(disk_errors2)}")
print('number of point fails > 1mm:', len(point_fails2))
print('number of cyl fails > 1mm:', len(syl_fails2))
print('number of disk fails > 1mm:', len(disk_fails2))

In [ ]:
# log histograms sim_2
fig2, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[9,3], dpi=100)
ax1.set_yscale('log')
ax1.hist(point_errors2, bins=20)
ax1.set_title('Point electrode errors')
ax1.tick_params('x', labelrotation=30)
ax1.set_ylabel('# count')
mark_subplots(ax1, 'A')
ax2.set_yscale('log')
ax2.hist(syl_errors2, bins=20)
ax2.tick_params('x', labelrotation=30)
ax2.set_xlabel('error (µm)')
ax2.set_title('Cylinder electrode errors')
mark_subplots(ax2, 'B')
ax3.set_yscale('log')
ax3.hist(disk_errors2, bins=20)
ax3.tick_params('x', labelrotation=30)
ax3.set_title('Disk electrode errors')
mark_subplots(ax3, 'C')

plt.tight_layout()

fig_folder = 'sim_2_rerun_tot'
fig_name = 'histograms_sim_2_log'
fig2.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')
print(f"Mean error(point): {np.mean(point_errors2)}, Median error(point): {np.median(point_errors2)}")
print(f"Mean error(syl): {np.mean(syl_errors2)}, Median error(syl): {np.median(syl_errors2)}")
print(f"Mean error(disk): {np.mean(disk_errors2)}, Median error(disk): {np.median(disk_errors2)}")
print('number of point fails > 1mm:', len(point_fails2))
print('number of cyl fails > 1mm:', len(syl_fails2))
print('number of disk fails > 1mm:', len(disk_fails2))

In [ ]:
np.savetxt('sim_2_point_errors2_rerun_tot.txt', point_errors2)
np.savetxt('sim_2_disk_errors2_rerun_tot.txt', disk_errors2)
np.savetxt('sim_2_syl_errors2_rerun_tot.txt', syl_errors2)

In [ ]:
#-----------sim_3-----------------------------
# comparing 5 disk sets(set pos), 20 point elecs centered at disks(set pos), 
# 20 point elecs(rd pos)and 20 disks(rd pos) 

n_points_disk = 1000
n_points_point = 1
orig_p = np.array([1., 1., 1.]) * 1e8
n_elecs = 20
n_elecs_set = 5
n_elecs_disk = n_elecs_set*4
num_trials = 100
box_width = 10000
x0 = -box_width / 2
x1 = box_width / 2
set_width = 2000 #disk_radius*2
disk_radius = 500
set_radius = set_width/2
elec_set_center_positions = np.array([(4000, -4000, -2000), (-4000, 4000, 1500), (4000, 4000, 4000),
                                      (-4000, -4000, 2000), (0, -4000, -4000)])#0, -4000, -4000
bounds = ((x0*1.2, x1*1.2), (x0*1.2, x1*1.2), (x0*1.2, x1*1.2))

point_errors3 = np.zeros(num_trials)
rd_point_errors3 = np.zeros(num_trials)
disk_errors3 = np.zeros(num_trials)
rd_disk_errors3 = np.zeros(num_trials)

point_fails3 = []
rd_point_fails = []
disk_fails3 = []
rd_disk_fails = []

for trial_idx in range(num_trials):
    set_coords, cs = g_d_s(disk_radius, set_radius, n_points_disk, n_elecs_set, elec_set_center_positions)
    orig_loc = np.random.uniform(x0, x1, size=(3))
    point_elec_locs  = cs
    V_e_measured_point = dipole_potential(orig_loc, point_elec_locs, orig_p, n_points_point, n_elecs)
    
    retry = True
    point_counter = 0
    while retry:
        
        dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)
        point_opt = minimize(dipole_potential_min, dipole_pos0,
                        options = {'maxiter': 50000., 'maxls': 100, 'gtol': 1e-12},
                        method='L-BFGS-B',
                        args =(point_elec_locs, V_e_measured_point, orig_p, n_points_point, n_elecs), 
                        bounds=bounds, tol = 1e-12#((x0, x1), (x0, x1), (x0, x1))
                        )
        V_e_best_guess_point = dipole_potential(point_opt.x, point_elec_locs, orig_p, n_points_point, n_elecs)
        rel_error_point = np.mean(np.abs((V_e_measured_point - V_e_best_guess_point) / V_e_measured_point))

        if (rel_error_point < 0.05) or (point_counter > 100):
            # If it doesn't work well, we just retry with a different initial guess
            retry = False

        point_counter += 1
    point_errors3[trial_idx] = np.sqrt(np.sum((orig_loc - point_opt.x)**2))
    if point_errors3[trial_idx] > 1000:
        point_fails3.append([point_elec_locs, orig_loc, point_errors3[trial_idx], point_opt.x, V_e_best_guess_point, 
                            V_e_measured_point, dipole_pos0, point_opt])
        
for trial_idx in range(num_trials):
    orig_loc = np.random.uniform(x0, x1, size=(3))
    rd_disk_elec_locs  = gen_random_disk_set(n_elecs, disk_radius, n_points_disk,
                                             (x0-disk_radius), (x1-disk_radius))
    V_e_measured_rd_disk = dipole_potential(orig_loc, rd_disk_elec_locs, orig_p, 
                                            n_points_disk, n_elecs)
    
    retry = True
    rd_disk_counter = 0
    while retry:
        
        dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)
        rd_disk_opt = minimize(dipole_potential_min, dipole_pos0,
                        options = {'maxiter': 50000., 'maxls': 100, 'gtol': 1e-12},
                        method='L-BFGS-B',
                        args =(rd_disk_elec_locs, V_e_measured_rd_disk, orig_p, n_points_disk, n_elecs), 
                        bounds=bounds, tol = 1e-12#((x0, x1), (x0, x1), (x0, x1))
                        )
        V_e_best_guess_rd_disk = dipole_potential(rd_disk_opt.x, rd_disk_elec_locs, orig_p, 
                                                  n_points_disk, n_elecs)
        rel_error_rd_disk = np.mean(np.abs((V_e_measured_rd_disk - V_e_best_guess_rd_disk) / V_e_measured_rd_disk))

        if (rel_error_rd_disk < 0.05) or (rd_disk_counter > 100):
            # If it doesn't work well, we just retry with a different initial guess
            retry = False

        rd_disk_counter += 1
    rd_disk_errors3[trial_idx] = np.sqrt(np.sum((orig_loc - rd_disk_opt.x)**2))
    if rd_disk_errors3[trial_idx] > 1000:
        rd_disk_fails.append([rd_disk_elec_locs, orig_loc, rd_disk_errors3[trial_idx], rd_disk_opt.x, V_e_best_guess_rd_disk, 
                            V_e_measured_rd_disk, dipole_pos0, rd_disk_opt])
    
for trial_idx in range(num_trials):
    orig_loc = np.random.uniform(x0, x1, size=(3))
    rd_point_elec_locs  = np.random.uniform(x0, x1, size=(n_elecs, 3))
    V_e_measured_rd_point = dipole_potential(orig_loc, rd_point_elec_locs, orig_p, n_points_point, n_elecs)
    
    retry = True
    rd_point_counter = 0
    while retry:
        
        dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)
        rd_point_opt = minimize(dipole_potential_min, dipole_pos0,
                        options = {'maxiter': 50000., 'maxls': 100, 'gtol': 1e-12},
                        method='L-BFGS-B',
                        args =(rd_point_elec_locs, V_e_measured_rd_point, orig_p, n_points_point, n_elecs), 
                        bounds=bounds, tol = 1e-12#((x0, x1), (x0, x1), (x0, x1))
                        )
        V_e_best_guess_rd_point = dipole_potential(rd_point_opt.x, rd_point_elec_locs, orig_p, n_points_point, n_elecs)
        rel_error_rd_point = np.mean(np.abs((V_e_measured_rd_point - V_e_best_guess_rd_point) / V_e_measured_rd_point))

        if (rel_error_rd_point < 0.05) or (rd_point_counter > 100):
            # If it doesn't work well, we just retry with a different initial guess
            retry = False

        rd_point_counter += 1
    rd_point_errors3[trial_idx] = np.sqrt(np.sum((orig_loc - rd_point_opt.x)**2))
    if rd_point_errors3[trial_idx] > 1000:
        rd_point_fails.append([rd_point_elec_locs, orig_loc, rd_point_errors3[trial_idx], rd_point_opt.x,
                               V_e_best_guess_rd_point, V_e_measured_rd_point, dipole_pos0, rd_point_opt])
        
for trial_idx in range(num_trials):
    orig_loc = np.random.uniform(x0, x1, size=(3))
    set_coords, cs = g_d_s(disk_radius, set_radius, n_points_disk, n_elecs_set, elec_set_center_positions)
    disk_elec_locs  = set_coords
    V_e_measured_disk = dipole_potential(orig_loc, disk_elec_locs, orig_p, n_points_disk, n_elecs_disk)
    
    retry = True
    disk_counter = 0
    while retry:
        dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)
        disk_opt = minimize(dipole_potential_min, dipole_pos0,
                        options = {'maxiter': 50000., 'maxls': 100, 'gtol': 1e-12},
                        method='L-BFGS-B',
                        args =(disk_elec_locs, V_e_measured_disk, orig_p, n_points_disk, n_elecs_disk), 
                        bounds=bounds, tol = 1e-12#((x0, x1), (x0, x1), (x0, x1))
                        )
        V_e_best_guess_disk = dipole_potential(disk_opt.x, disk_elec_locs, orig_p, n_points_disk, n_elecs_disk)
        rel_error_disk = np.mean(np.abs((V_e_measured_disk - V_e_best_guess_disk) / V_e_measured_disk))

        if (rel_error_disk < 0.05) or (disk_counter > 100):
                # If it doesn't work well, we just retry with a different initial guess
            retry = False

        disk_counter += 1
    disk_errors3[trial_idx] = np.sqrt(np.sum((orig_loc - disk_opt.x)**2))
    if disk_errors3[trial_idx] > 1000:
        disk_fails3.append([disk_elec_locs, orig_loc, disk_errors3[trial_idx], disk_opt.x, V_e_best_guess_disk, 
                            V_e_measured_disk, dipole_pos0, disk_opt])


print('number of point fails > 1mm:', len(point_fails3))
for i in range(len(point_fails3)):
    print('error:', point_fails3[i][2])
    print(point_fails3[i][2], 'orig_loc:', point_fails3[i][1], 'opt_x:',point_fails3[i][3], 'init guess:',point_fails3[i][6],
          'V_e best guess:',point_fails3[i][4], 'V_e measured:',point_fails3[i][5], 'elec positions:', point_fails3[i][0])
    fig_name = f'point_fails3_{i+1}'
    fig_folder = 'sim_3_rerun_tot'
    plot_res(point_fails3[i][0], point_fails3[i][1], point_fails3[i][3],
            f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}',
            f'Random disk fail number {i+1}')

print('number of rd disk fails > 1mm:', len(rd_disk_fails))
for i in range(len(rd_disk_fails)):
    print('error:',rd_disk_fails[i][2], 'orig_loc:', rd_disk_fails[i][1], 'opt_x:',
          rd_disk_fails[i][3], 'init guess:',rd_disk_fails[i][6],'V_e best guess:',
          rd_disk_fails[i][4], 'V_e measured:',rd_disk_fails[i][5], 'elec positions:', rd_disk_fails[i][0])
    fig_name = f'rd_disk_fails3_{i+1}'
    fig_folder = 'sim_3_rerun_tot'
    plot_res(rd_disk_fails[i][0], rd_disk_fails[i][1], rd_disk_fails[i][3],
            f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}',
            f'Random disk fail number {i+1}')
    
print('number of rd_point fails > 1mm:', len(rd_point_fails))    
for i in range(len(rd_point_fails)):
    print('error:',rd_point_fails[i][2], 'orig_loc:', rd_point_fails[i][1], 'opt_x:',
          rd_point_fails[i][3], 'init guess:',rd_point_fails[i][6],'V_e best guess:',rd_point_fails[i][4], 
          'V_e measured:',rd_point_fails[i][5], 'elec positions:', rd_point_fails[i][0])
    fig_name = f'rd_point_fails3_{i+1}'
    fig_folder = 'sim_3_rerun_tot'
    plot_res(rd_point_fails[i][0], rd_point_fails[i][1], rd_point_fails[i][3],
            f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}',
            f'Random point fail number {i+1}')

print('number of disk fails > 1mm:', len(disk_fails3))
for i in range(len(disk_fails3)):
    print('error:',disk_fails3[i][2], 'orig_loc:', disk_fails3[i][1], 'opt_x:',disk_fails3[i][3], 'init guess:',disk_fails3[i][6],
          'V_e best guess:',disk_fails3[i][4], 'V_e measured:',disk_fails3[i][5], 'elec positions:', disk_fails3[i][0])
    fig_name = f'disk_fails3_{i+1}'
    fig_folder = 'sim_3_rerun_tot'
    plot_res(disk_fails3[i][0], disk_fails3[i][1], disk_fails3[i][3],
            f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}',
            f'Disk fail number {i+1}')   


In [ ]:
# histograms sim 3
fig3, ((ax1, ax2),( ax3, ax4)) = plt.subplots(2, 2, figsize=[8,6], dpi=100)
ax1.hist(point_errors3, bins=20)
ax1.set_title('Point electrode errors')
ax1.tick_params('x', labelrotation=30)
ax1.set_ylabel('# count')
ax1.set_xlabel('error (µm)')
mark_subplots(ax1, 'A')

ax2.hist(rd_point_errors3, bins=20)
ax2.tick_params('x', labelrotation=30)
ax2.set_xlabel('error (µm)')
ax2.set_title('Random point electrode errors')
ax2.set_ylabel('# count')
mark_subplots(ax2, 'B')

ax3.hist(disk_errors3, bins=20)
ax3.tick_params('x', labelrotation=30)
ax3.set_title('Disk electrode errors')
ax3.set_ylabel('# count')
ax3.set_xlabel('error (µm)')
mark_subplots(ax3, 'C')

ax4.hist(rd_disk_errors3, bins=20)
ax4.tick_params('x', labelrotation=30)
ax4.set_title('Random disk electrode errors')
ax4.set_ylabel('# count')
ax4.set_xlabel('error (µm)')
mark_subplots(ax4, 'D')

plt.tight_layout()

fig_folder = 'sim_3_rerun_tot'
fig_name = 'histograms_sim_3'
fig3.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')
print(f"Mean error(point): {np.mean(point_errors3)}, Median error(point): {np.median(point_errors3)}")
print(f"Mean error(rd point): {np.mean(rd_point_errors3)}, Median error(rd point): {np.median(rd_point_errors3)}")
print(f"Mean error(disk): {np.mean(disk_errors3)}, Median error(disk): {np.median(disk_errors3)}")
print(f"Mean error(rd disk): {np.mean(rd_disk_errors3)}, Median error(disk): {np.median(rd_disk_errors3)}")

In [ ]:
# log histograms sim 3
fig3, ((ax1, ax2),( ax3, ax4)) = plt.subplots(2, 2, figsize=[8,6], dpi=100)
ax1.set_yscale('log')
ax1.hist(point_errors3, bins=20)
ax1.set_title('Point electrode errors')
ax1.tick_params('x', labelrotation=30)
ax1.set_ylabel('# count')
ax1.set_xlabel('error (µm)')
mark_subplots(ax1, 'A')

ax2.set_yscale('log')
ax2.hist(rd_point_errors3, bins=20)
ax2.tick_params('x', labelrotation=30)
ax2.set_xlabel('error (µm)')
ax2.set_title('Random point electrode errors')
ax2.set_ylabel('# count')
mark_subplots(ax2, 'B')

ax3.set_yscale('log')
ax3.hist(disk_errors3, bins=20)
ax3.tick_params('x', labelrotation=30)
ax3.set_title('Disk electrode errors')
ax3.set_ylabel('# count')
ax3.set_xlabel('error (µm)')
mark_subplots(ax3, 'C')

ax4.set_yscale('log')
ax4.hist(rd_disk_errors3, bins=20)
ax4.tick_params('x', labelrotation=30)
ax4.set_title('Random disk electrode errors')
ax4.set_ylabel('# count')
ax4.set_xlabel('error (µm)')
mark_subplots(ax4, 'D')

plt.tight_layout()

fig_folder = 'sim_3_rerun_tot'
fig_name = 'histograms_sim_3_log'
fig3.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')
print(f"Mean error(point): {np.mean(point_errors3)}, Median error(point): {np.median(point_errors3)}")
print(f"Mean error(rd point): {np.mean(rd_point_errors3)}, Median error(rd point): {np.median(rd_point_errors3)}")
print(f"Mean error(disk): {np.mean(disk_errors3)}, Median error(disk): {np.median(disk_errors3)}")
print(f"Mean error(rd disk): {np.mean(rd_disk_errors3)}, Median error(disk): {np.median(rd_disk_errors3)}")

In [ ]:
np.savetxt('sim_3_point_errors3_rerun_tot.txt', point_errors3)
np.savetxt('sim_3_rd_point_errors3_rerun_tot.txt', rd_point_errors3)
np.savetxt('sim_3_disk_errors3_rerun_tot.txt', disk_errors3)
np.savetxt('sim_3_rd_disk_errors3_rerun_tot.txt', rd_disk_errors3)

In [ ]:
# ------------------------sim_8-----------------------------------------------
# let n_elecs grow for pointelectrodes to show that more elecs gives more accurate dipole loc
num_point_elecs = np.array([1,2,3,4,5,6,7])
n_points_point = 1
orig_p = np.array([1., 1., 1.]) * 1e8
num_trials = 100
box_width = 10000
x0 = -box_width / 2
x1 = box_width / 2
point_errors_tot_sim_8 = np.zeros(shape=(num_trials, len(num_point_elecs)))

for i, n_elecs in enumerate(num_point_elecs): 
    #point_errors = np.zeros(num_trials)
    
    for trial_idx in range(num_trials):
        orig_loc = np.random.uniform(x0, x1, size=(3))
        point_elec_locs  = np.random.uniform(x0, x1, size=(n_elecs, 3))
        V_e_measured_point = dipole_potential(orig_loc, point_elec_locs, orig_p, n_points_point, n_elecs)
    
        retry = True
        counter = 0
        while retry:
            dipole_pos0 = np.random.uniform(x0, x1, size=(3))# (initial guess)
            point_opt = minimize(dipole_potential_min, dipole_pos0,
                        options = {'maxiter': 50000.},
                        method='L-BFGS-B',
                        args =(point_elec_locs, V_e_measured_point, orig_p, n_points_point, n_elecs), 
                        bounds=((x0, x1), (x0, x1), (x0, x1))
                        )
            V_e_best_guess_point = dipole_potential(point_opt.x, point_elec_locs, orig_p, n_points_point, n_elecs)
            rel_error_point = np.mean(np.abs((V_e_measured_point - V_e_best_guess_point) / V_e_measured_point))

            if (rel_error_point < 0.05) or (counter > 100):
                # If it doesn't work well, we just retry with a different initial guess
                retry = False

            counter += 1
        point_errors_tot_sim_8[trial_idx, i] = np.sqrt(np.sum((orig_loc - point_opt.x)**2))

In [ ]:
fig8 = plt.figure(figsize=[6, 4], dpi=100)
plt.xlabel('no point electrodes')
plt.ylabel('localization error (µm)')
for i in range(num_trials):
    plt.scatter(num_point_elecs,(point_errors_tot_sim_8[i]), s=1)  
fig_name = 'num_elecs_errors'
fig_folder = 'sim_8_rerun_tot'
fig8.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')

In [ ]:
fig8_2 = plt.figure(figsize=[6, 4], dpi=100)
plt.boxplot(point_errors_tot_sim_8)
plt.xlabel('no point electrodes')
plt.ylabel('localization error (µm)')
fig_name = 'num_elecs_errors_boxplot'
fig_folder = 'sim_8_rerun_tot'
fig8_2.savefig(f'C:\\Users\\SunRe_Admin\\OneDrive\\Skrivebord\\master\\{fig_folder}\\{fig_name}', bbox_inches='tight')

In [ ]:
np.savetxt('sim_8_point_errors_tot_sim_8_rerun_tot.txt', point_errors_tot_sim_8)